## TFT
- need to create feature like date and day of week. 

In [48]:
import xarray as xr
import ocf_blosc2
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# import matplotlib.pyplot as plt
# import plotly.express as px
import numpy as np


In [49]:
nwp_data = xr.open_dataset("../../../mnt/disks/gcp_data/nwp/ecmwf/UK_v2.zarr")
nwp_data

<xarray.Dataset> Size: 1TB
Dimensions:    (variable: 14, init_time: 5049, step: 49, latitude: 241,
                longitude: 301)
Coordinates:
  * init_time  (init_time) datetime64[ns] 40kB 2017-01-01 ... 2023-12-31T12:0...
  * latitude   (latitude) float64 2kB 60.0 59.95 59.9 59.85 ... 48.1 48.05 48.0
  * longitude  (longitude) float64 2kB -12.0 -11.95 -11.9 ... 2.9 2.95 3.0
  * step       (step) timedelta64[ns] 392B 00:00:00 01:00:00 ... 2 days 00:00:00
  * variable   (variable) <U5 280B 't2m' 'dswrf' 'dlwrf' ... 'v10' 'u100' 'v100'
Data variables:
    ECMWF_UK   (variable, init_time, step, latitude, longitude) float32 1TB ...

In [50]:
pv_data = xr.open_dataset("data_files/pv.netcdf", engine='h5netcdf')
pv_data

<xarray.Dataset> Size: 2GB
Dimensions:   (datetime: 387254)
Coordinates:
  * datetime  (datetime) datetime64[ns] 3MB 2018-01-01T06:05:00 ... 2021-10-2...
Data variables: (12/1311)
    10003     (datetime) float32 2MB ...
    10004     (datetime) float32 2MB ...
    10005     (datetime) float32 2MB ...
    10048     (datetime) float32 2MB ...
    10049     (datetime) float32 2MB ...
    10063     (datetime) float32 2MB ...
    ...        ...
    9870      (datetime) float32 2MB ...
    9871      (datetime) float32 2MB ...
    9902      (datetime) float32 2MB ...
    9903      (datetime) float32 2MB ...
    9960      (datetime) float32 2MB ...
    9989      (datetime) float32 2MB ...

- For 50 pv_sites, we are selecting 1500 dates. Means 75000
- Then we select 48 hours for each, but not all dates have 48 hours. 
- My dataset is 17046
- It means not all ss_ids, have 1500 datetime, with 48 hours gap. because i am selecting random after 48 hours gap.

In [51]:
## proof as data poitns are missing and not all ss_id ahve data for all hours in the dates. 


import pandas as pd

# Define the specific ss_id and the datetime range
ss_id = '6641'
start_datetime = pd.Timestamp('2021-10-27 19:00:00')
end_datetime = pd.Timestamp('2021-10-29 18:59:00')


hourly_pv_data = pv_data.sel(datetime=pv_data['datetime'].dt.minute == 0)
# Select the data for the specified range and ss_id
selected_data = hourly_pv_data.sel(datetime=slice(start_datetime, end_datetime))

# Check available datetimes
available_datetimes = selected_data['datetime'].values
generation_values = selected_data[ss_id].values

# Create a DataFrame to display the results
results_df = pd.DataFrame({
    'datetime': available_datetimes,
    'generation': generation_values
})

print(results_df)


             datetime  generation
0 2021-10-27 19:00:00         0.0
1 2021-10-27 20:00:00         0.0
2 2021-10-27 21:00:00         0.0
3 2021-10-27 22:00:00         0.0
4 2021-10-27 23:00:00         0.0


Conclusion is that not all points  have all hourly data between 2 days, so it is quite possible if i select a point, it might have only next 5 hours, and not all 48 hours. So dataset would not be optimal 25 ss_ids to 50 datetimes and 48 steps. Multiplication won't work, because there are too many missing data points. 

In [52]:
# add lat, long, tilt orientation
meta_data = pd.read_csv("data_files/metadata.csv")
meta_data.head()

,ss_id,latitude_rounded,longitude_rounded,llsoacd,orientation,tilt,kwp,operational_at
0,2405,53.53,-1.63,E01007430,180.0,35.0,3.36,2010-11-18
1,2406,54.88,-1.38,E01008780,315.0,30.0,1.89,2010-12-03
2,2407,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
3,2408,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
4,2409,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03


In [53]:
def get_48_hour_range(start_datetime):
    end_datetime = start_datetime + pd.Timedelta(hours=47, minutes=59)
    return start_datetime, end_datetime

def select_non_overlapping_datetimes(datetimes, num_selections, min_gap_hours):
    selected_datetimes = []
    available_datetimes = list(datetimes)

    for _ in range(num_selections):
        if not available_datetimes:
            break
        random_datetime = np.random.choice(available_datetimes)
        selected_datetimes.append(random_datetime)
        
        # Remove datetimes within the min_gap_hours range
        available_datetimes = [dt for dt in available_datetimes if dt > random_datetime + pd.Timedelta(hours=min_gap_hours)]

    return selected_datetimes

In [62]:

hourly_pv_data = pv_data.sel(datetime=pv_data['datetime'].dt.minute == 0)
datetimes = pd.to_datetime(hourly_pv_data['datetime'].values)

data_vars = list(pv_data.data_vars)
pv_sites_id = np.random.choice(data_vars, 50, replace=False)
print(pv_sites_id)

data_dict = {'ss_id': [], 'pv_datetime': [], 'generation' : []}

num_selections = 1500
min_gap_hours = 48

for ss_id in pv_sites_id:
    selected_datetimes = select_non_overlapping_datetimes(datetimes, num_selections, min_gap_hours)
    
    for start_datetime in selected_datetimes:
        start, end = get_48_hour_range(start_datetime)
        selected_data = hourly_pv_data.sel(datetime=slice(start, end))
        
        for dt, power in zip(selected_data['datetime'].values, selected_data[ss_id].values):
            data_dict['ss_id'].append(int(ss_id))
            data_dict['pv_datetime'].append(dt)
            data_dict['generation'].append(power)
                                           



['26906' '9763' '10704' '7359' '7890' '11628' '12314' '10003' '26883'
 '17693' '6862' '6971' '7748' '6504' '7636' '14394' '7276' '7910' '26890'
 '14786' '26828' '7440' '27043' '6094' '10548' '12648' '4158' '7845'
 '12644' '26813' '16215' '7905' '12847' '9368' '10585' '5444' '6896'
 '7393' '11752' '9866' '7291' '15851' '26772' '4127' '6667' '11153'
 '27057' '7661' '6801' '6093']


In [63]:


pv_df = pd.DataFrame(data_dict)
print(pv_df.shape)
pv_df = pv_df.dropna(subset={'generation'})

pv_df
print(pv_df.shape)


(15370, 3)
(10196, 3)


In [64]:
pv_df.head()

,ss_id,pv_datetime,generation
26,26906,2021-07-29 02:00:00,0.000000
27,26906,2021-07-29 03:00:00,0.000000
28,26906,2021-07-29 04:00:00,0.000000
29,26906,2021-07-29 05:00:00,6.474624
30,26906,2021-07-29 06:00:00,43.018799


In [58]:
# pv_df.to_csv("Try.csv")

In [65]:
pv_sites_id = [int(id) for id in pv_sites_id]  
# pv_sites_id

In [66]:
pv_site_dict = {'ss_id':[], "lat":[], "long": [], 'tilt':[], 'orientation':[]}

for id in pv_sites_id:
    row = meta_data[meta_data['ss_id'] == id]
    if not row.empty:
        pv_site_dict['ss_id'].append(id)
        pv_site_dict['lat'].append(row['latitude_rounded'].values[0])
        pv_site_dict['long'].append(row['longitude_rounded'].values[0])
        pv_site_dict['tilt'].append(row['tilt'].values[0])
        pv_site_dict['orientation'].append(row['orientation'].values[0])
    else:
        print('row empty')
        


print(pv_site_dict)
meta_site_df = pd.DataFrame.from_dict(pv_site_dict)
# meta_site_df

{'ss_id': [26906, 9763, 10704, 7359, 7890, 11628, 12314, 10003, 26883, 17693, 6862, 6971, 7748, 6504, 7636, 14394, 7276, 7910, 26890, 14786, 26828, 7440, 27043, 6094, 10548, 12648, 4158, 7845, 12644, 26813, 16215, 7905, 12847, 9368, 10585, 5444, 6896, 7393, 11752, 9866, 7291, 15851, 26772, 4127, 6667, 11153, 27057, 7661, 6801, 6093], 'lat': [53.52, 53.27, 57.37, 51.03, 54.63, 51.89, 55.83, 53.36, 50.39, 53.32, 53.17, 51.55, 51.07, 50.34, 57.68, 55.82, 50.77, 50.54, 51.35, 52.08, 51.1, 55.06, 53.66, 52.45, 54.58, 51.67, 57.23, 54.91, 51.58, 51.28, 53.85, 54.08, 53.56, 55.82, 51.7, 50.72, 51.14, 55.96, 55.95, 53.09, 52.43, 54.18, 51.52, 51.57, 57.33, 53.72, 51.52, 50.35, 51.57, 56.43], 'long': [-2.93, -1.34, -2.09, -2.86, -3.56, -2.11, -4.05, -2.84, -4.16, -3.48, -0.77, -1.84, -4.1, -4.8, -1.93, -3.96, -4.13, -3.62, -0.8, -4.49, -3.01, -3.58, -2.08, 1.42, -1.31, -3.81, -2.14, -2.95, -2.95, -2.39, -3.05, -0.21, -2.37, -4.36, -3.01, -3.5, -2.98, -2.78, -4.55, -1.24, -1.96, -3.11, -2.54, -1

In [67]:
combined_df = pd.merge(pv_df, meta_site_df, on='ss_id', how='inner')
combined_df

,ss_id,pv_datetime,generation,lat,long,tilt,orientation
0,26906,2021-07-29 02:00:00,0.000000,53.52,-2.93,26.0,225.0
1,26906,2021-07-29 03:00:00,0.000000,53.52,-2.93,26.0,225.0
2,26906,2021-07-29 04:00:00,0.000000,53.52,-2.93,26.0,225.0
3,26906,2021-07-29 05:00:00,6.474624,53.52,-2.93,26.0,225.0
4,26906,2021-07-29 06:00:00,43.018799,53.52,-2.93,26.0,225.0
...,...,...,...,...,...,...,...
10191,6093,2021-10-27 19:00:00,0.000000,56.43,-2.95,45.0,180.0
10192,6093,2021-10-27 20:00:00,0.000000,56.43,-2.95,45.0,180.0
10193,6093,2021-10-27 21:00:00,0.000000,56.43,-2.95,45.0,180.0
10194,6093,2021-10-27 22:00:00,0.000000,56.43,-2.95,45.0,180.0


In [68]:
# combined_df.to_csv("test.csv")

In [69]:
# test = pd.read_csv("test.csv")
# test.head()

In [70]:
combined_df['pv_datetime'] = pd.to_datetime(combined_df['pv_datetime'])

# Extract date and hour from pv_datetime
combined_df['pv_date'] = combined_df['pv_datetime'].dt.date
combined_df['pv_hour'] = combined_df['pv_datetime'].dt.hour
combined_df

,ss_id,pv_datetime,generation,lat,long,tilt,orientation,pv_date,pv_hour
0,26906,2021-07-29 02:00:00,0.000000,53.52,-2.93,26.0,225.0,2021-07-29,2
1,26906,2021-07-29 03:00:00,0.000000,53.52,-2.93,26.0,225.0,2021-07-29,3
2,26906,2021-07-29 04:00:00,0.000000,53.52,-2.93,26.0,225.0,2021-07-29,4
3,26906,2021-07-29 05:00:00,6.474624,53.52,-2.93,26.0,225.0,2021-07-29,5
4,26906,2021-07-29 06:00:00,43.018799,53.52,-2.93,26.0,225.0,2021-07-29,6
...,...,...,...,...,...,...,...,...,...
10191,6093,2021-10-27 19:00:00,0.000000,56.43,-2.95,45.0,180.0,2021-10-27,19
10192,6093,2021-10-27 20:00:00,0.000000,56.43,-2.95,45.0,180.0,2021-10-27,20
10193,6093,2021-10-27 21:00:00,0.000000,56.43,-2.95,45.0,180.0,2021-10-27,21
10194,6093,2021-10-27 22:00:00,0.000000,56.43,-2.95,45.0,180.0,2021-10-27,22


## Matching pv data with NWP
Need to match pv_datetime, with nwp, split pv_datetime to date and time seperatly, match pv_date with nwp init_time, and pv_hour with nwp steps. 

- 12 is midnight, so need to match with previous time.  using `backfill`

In [71]:
dataframes = []

In [46]:
# # approach is to match exact date and time with init_time, previous one,a nd then get all the next 48 steps from nwp data. 
# # take from pv_datetime, date and hour to match with nwp_init time, now go to next 48 rows in combined_dataset for the next 48 steps of pv_datetime
# # now match these 48 steps with 49 steps available in nwp data fro each init_time. 
# # and get the valeus of aviables for the next 48 steps and append to combine dataset. 

# # now conider that not all points have 48 hours continous data. so blocks can not be of 48 hours. 

# dataframes = []

# # Iterate over the combined_df in steps of 48 to process each 48-hour block
# for start_idx in range(0, len(combined_df), 48):
#     block = combined_df.iloc[start_idx:start_idx + 48]
#     if len(block) < 48 or not is_continuous(block):
#         print("Not a continuous 48-hour block:", block.iloc[0]['pv_datetime'])
#         continue   # Skip blocks that are not full 48-hour periods

#     ss_id = block.iloc[0]['ss_id']
#     init_time = block.iloc[0]['pv_datetime']
#     lat = block.iloc[0]['lat']
#     lon = block.iloc[0]['long']
#     generation_values = block['generation'].values
#     pv_datetimes = block['pv_datetime'].values
#     tilt = block.iloc[0]['tilt']
#     orientation = block.iloc[0]['orientation']

#     # Match pv_datetime to the nearest previous init_time
#     init_time_sel = nwp_data.sel(init_time=init_time, method="backfill")
    
#     # Retrieve the 48 steps of forecast data starting from the matched init_time
#     data_sel = init_time_sel.sel(step=slice(pd.Timedelta(hours=0), pd.Timedelta(hours=47)))
#     data_sel = data_sel.sel(latitude=lat, method="nearest") \
#                        .sel(longitude=lon, method="nearest")
    
#     # Convert to DataFrame and pivot
#     data_df = data_sel.to_dataframe().reset_index()
#     pivot_df = data_df.pivot_table(index=['init_time', 'step'], columns='variable', values='ECMWF_UK').reset_index()

#     # Add the combined dataset columns
#     for i in range(48):
#         pivot_df.loc[i, 'ss_id'] = ss_id
#         pivot_df.loc[i, 'pv_datetime'] = pv_datetimes[i]
#         pivot_df.loc[i, 'generation'] = generation_values[i]

#     dataframes.append(pivot_df)
    



In [72]:
len(dataframes)

0

In [ ]:
# print(len(dataframes))
# # Combine all dataframes into one
# forecast_df = pd.concat(dataframes, ignore_index=True)

# # Display the final DataFrame
# print(forecast_df)

In [78]:
# approach is to match exact date of pvdate with init time date, adn then match exact time of pv hour with step hour. 

dataframes = []

for index, row in combined_df.iterrows():

    ss_id = row['ss_id']

    # Match pv_date to init_time
    init_time_sel = nwp_data.sel(init_time=row['pv_datetime'], method="backfill")
    
    # Match pv_hour to step
    step_sel = init_time_sel.sel(step=pd.Timedelta(hours=row['pv_hour']), method="nearest")
    
    data_sel = step_sel.sel(latitude=row['lat'], method="nearest") \
                       .sel(longitude=row['long'], method="nearest")
    

    data_df = data_sel.to_dataframe().reset_index()
    pivot_df = data_df.pivot_table(index=['init_time', 'step'], columns='variable', values='ECMWF_UK').reset_index()

    pivot_df['ss_id'] = ss_id
    pivot_df['pv_datetime'] = row['pv_datetime']
    pivot_df['generation'] = row['generation']
    pivot_df['pv_hour'] = row['pv_hour']
    pivot_df['tilt'] = row['tilt']
    pivot_df['orientation'] = row['orientation']
    pivot_df['lat'] = row['lat']
    pivot_df['long'] = row['long']
    

    dataframes.append(pivot_df)

In [79]:
# dataframes
print(len(dataframes))

10196


In [80]:
final_df = pd.concat(dataframes, ignore_index=True)
desired_order = ['ss_id', 'init_time', 'step', 'generation', 'pv_datetime', 'pv_hour', 'lat', 'long', 'tilt', 'orientation', 'dlwrf', 'dswrf', 'duvrs', 'hcc', 'lcc', 'mcc', 'sde', 'sr', 't2m', 'tcc', 'u10', 'u100', 'v10', 'v100']
final_df = final_df[desired_order]
print(final_df.shape)

(10196, 24)


In [81]:
final_df

variable,ss_id,init_time,step,generation,pv_datetime,pv_hour,lat,long,tilt,orientation,...,lcc,mcc,sde,sr,t2m,tcc,u10,u100,v10,v100
0,26906,2021-07-29 12:00:00,0 days 02:00:00,0.000000,2021-07-29 02:00:00,2,53.52,-2.93,26.0,225.0,...,0.999420,0.958649,0.0,3.193600e+05,289.200439,0.999971,9.506679,12.843964,1.211929,1.569199
1,26906,2021-07-29 12:00:00,0 days 03:00:00,0.000000,2021-07-29 03:00:00,3,53.52,-2.93,26.0,225.0,...,0.991364,0.983948,0.0,4.513280e+05,289.295410,0.998687,8.772396,11.869049,0.832458,1.075439
2,26906,2021-07-29 12:00:00,0 days 04:00:00,0.000000,2021-07-29 04:00:00,4,53.52,-2.93,26.0,225.0,...,0.756744,0.946167,0.0,7.316480e+05,289.566162,0.999325,7.909646,10.661621,0.461304,0.630722
3,26906,2021-07-29 12:00:00,0 days 05:00:00,6.474624,2021-07-29 05:00:00,5,53.52,-2.93,26.0,225.0,...,0.548706,0.772430,0.0,9.912784e+05,289.593018,0.987131,6.859680,9.287048,-0.223587,-0.262619
4,26906,2021-07-29 12:00:00,0 days 06:00:00,43.018799,2021-07-29 06:00:00,6,53.52,-2.93,26.0,225.0,...,0.144135,0.644653,0.0,1.281776e+06,289.502686,0.746552,6.261518,8.631668,-0.405579,-0.522446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10191,6093,2021-10-28 00:00:00,0 days 19:00:00,0.000000,2021-10-27 19:00:00,19,56.43,-2.95,45.0,180.0,...,0.009705,0.094360,0.0,1.536000e+03,282.955566,1.000000,0.967721,1.983219,0.959087,1.334324
10192,6093,2021-10-28 00:00:00,0 days 20:00:00,0.000000,2021-10-27 20:00:00,20,56.43,-2.95,45.0,180.0,...,0.068848,0.570282,0.0,1.536000e+03,281.830322,1.000000,0.318899,0.754925,1.661366,3.263248
10193,6093,2021-10-28 00:00:00,0 days 21:00:00,0.000000,2021-10-27 21:00:00,21,56.43,-2.95,45.0,180.0,...,0.275391,0.967896,0.0,1.536000e+03,283.080811,1.000000,-0.390615,0.065497,0.486814,2.084058
10194,6093,2021-10-28 00:00:00,0 days 22:00:00,0.000000,2021-10-27 22:00:00,22,56.43,-2.95,45.0,180.0,...,0.700439,0.984222,0.0,1.536000e+03,282.872070,1.000000,-0.525719,-1.159500,-0.302175,0.762788


In [82]:
final_df.columns

Index(['ss_id', 'init_time', 'step', 'generation', 'pv_datetime', 'pv_hour',
       'lat', 'long', 'tilt', 'orientation', 'dlwrf', 'dswrf', 'duvrs', 'hcc',
       'lcc', 'mcc', 'sde', 'sr', 't2m', 'tcc', 'u10', 'u100', 'v10', 'v100'],
      dtype='object', name='variable')

In [83]:
final_df.to_csv("result_data/final_data_tft2.csv")

In [ ]:
## plots of generation and weather variables to see the trends
## add titl and orientation in dataset

## Wrong Approach
Instead of matching each datetime and hour with every datetime and steps. 
We need to select only one pv datetime to one nwp init_time, and then use next 48 hours from it.
Not exact matching with datetime. 

We need to get 48 next hours prediction. 

Try this as well, but focus more on model training. 
